In [1]:
import tensorflow as tf
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
import scipy.io
from scipy.misc import imresize
import matplotlib.pyplot as plt
from my_utils import *

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
data_path = os.path.join('Sitting','img')
label_path = os.path.join('Sitting','masks')
data_names = sorted(os.listdir(data_path))
label_names = sorted(os.listdir(label_path))
data_addr = []
for i in range(len(data_names)):
    data_addr.append(os.path.join('Sitting/img',data_names[i]))

label_addr = []
for i in range(len(label_names)):
    label_addr.append(os.path.join('Sitting/masks',label_names[i]))
The_Main_List = pd.DataFrame(data_names,columns = ['data_name'])
temp_frame1 = pd.DataFrame(label_names,columns=['label_name'])
temp_frame2 = pd.DataFrame(data_addr,columns=['data_addr'])
temp_frame3 = pd.DataFrame(label_addr,columns=['label_addr'])
The_Main_List = pd.concat([The_Main_List,temp_frame1],axis=1)
The_Main_List = pd.concat([The_Main_List,temp_frame2],axis=1)
The_Main_List = pd.concat([The_Main_List,temp_frame3],axis=1)
The_Main_List.head()


,data_name,label_name,data_addr,label_addr
0,image0.jpg,image0.mat,Sitting/img\image0.jpg,Sitting/masks\image0.mat
1,image1.jpg,image1.mat,Sitting/img\image1.jpg,Sitting/masks\image1.mat
2,image10.jpg,image10.mat,Sitting/img\image10.jpg,Sitting/masks\image10.mat
3,image100.jpg,image100.mat,Sitting/img\image100.jpg,Sitting/masks\image100.mat
4,image101.jpg,image101.mat,Sitting/img\image101.jpg,Sitting/masks\image101.mat


In [3]:
The_Main_List.to_csv('sitting_data_table.csv')

In [3]:
print('--Parsing Config File')
params = process_config('config.cfg')

--Parsing Config File


In [4]:
model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'], 
                       nModules=params['nmodules'], 
                       nLow=params['nlow'], outputDim=params['num_joints'], 
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'], 
                       lear_rate=params['learning_rate'], 
                       decay=params['learning_rate_decay'], 
                       decay_step=params['decay_step'], 
                       name=params['name'], 
                       logdir_train=params['log_dir_train'], 
                       logdir_test=params['log_dir_test'], 
                       tiny= params['tiny'], w_loss=params['weighted_loss'] , 
                       joints= params['joint_list'],modif=False)
model.generate_model()

CREATE MODEL:
---Inputs : Done (0 sec.)


---Graph : Done (11 sec.)
---Loss : Done (0 sec.)


In [5]:
saver = tf.train.Saver()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

path = "trained_nets/net_with_pascal_Mar_27"
full_path = os.path.join(path,'save_net.ckpt')
saver.restore(sess,full_path)

INFO:tensorflow:Restoring parameters from trained_nets/net_with_pascal_Mar_27\save_net.ckpt


In [6]:
pascal_data = pd.read_csv('pacal_data_inform.csv')
pascal_data.head()

,Unnamed: 0,jpg_path,mat_path
0,0,VOC2010/JPEGImages\2008_000003.jpg,VOC2010/all_person_masks\2008_000003.mat
1,1,VOC2010/JPEGImages\2008_000008.jpg,VOC2010/all_person_masks\2008_000008.mat
2,2,VOC2010/JPEGImages\2008_000023.jpg,VOC2010/all_person_masks\2008_000023.mat
3,3,VOC2010/JPEGImages\2008_000026.jpg,VOC2010/all_person_masks\2008_000026.mat
4,4,VOC2010/JPEGImages\2008_000032.jpg,VOC2010/all_person_masks\2008_000032.mat


In [17]:
i = 3431
img = image.img_to_array(image.load_img(pascal_data['jpg_path'][i]))
label = scipy.io.loadmat(pascal_data['mat_path'][i])['single_map']
cut1,cut2,cut3 = random_put_into_256map(img,label)
plt.figure()
plt.imshow(cut3[:,:,0:3]/256)
plt.show()
test_image = np.expand_dims(cut3[:,:,0:3],0)
output = sess.run(model.output,feed_dict={model.img:test_image})
out = output[0,3]
seg = np.argmax(out,axis=2)
plt.figure()
plt.imshow(seg)
plt.show()

In [18]:
acc_list=[]
iou_list=[]
for i in range(3450,3500):
    img = image.img_to_array(image.load_img(pascal_data['jpg_path'][i]))
    label = scipy.io.loadmat(pascal_data['mat_path'][i])['single_map']
    cut1,cut2,cut3 = random_put_into_256map(img,label)
    test_image = np.expand_dims(cut3[:,:,0:3],0)
    output = sess.run(model.output,feed_dict={model.img:test_image})
    out = output[0,3]
    seg = np.argmax(out,axis=2)
    temp_acc = cal_acc(seg,my_scale(cut3[:,:,3]))
    if temp_acc != 0:
        acc_list.append(temp_acc)
    temp_iou = cal_iou(seg,my_scale(cut3[:,:,3]))
    if temp_iou != 0:
        iou_list.append(temp_iou)

In [20]:
np.mean(iou_list)

0.11429012744481745

In [10]:
i = 46
img = image.img_to_array(image.load_img(The_Main_List['data_addr'][i]))
label = scipy.io.loadmat(The_Main_List['label_addr'][i])['M']
cut1,cut2,cut3 = random_put_into_256map(img,label)
plt.figure()
plt.imshow(cut3[:,:,0:3]/256)
plt.show()
test_image = np.expand_dims(cut3[:,:,0:3],0)
output = sess.run(model.output,feed_dict={model.img:test_image})
out = output[0,3]
seg = np.argmax(out,axis=2)
plt.figure()
plt.imshow(seg)
plt.show()

In [26]:
acc_list=[]
iou_list=[]
for i in range(20):
    img = image.img_to_array(image.load_img(The_Main_List['data_addr'][i]))
    label = scipy.io.loadmat(The_Main_List['label_addr'][i])['M']
    cut1,cut2,cut3 = random_put_into_256map(img,label)
    test_image = np.expand_dims(cut3[:,:,0:3],0)
    output = sess.run(model.output,feed_dict={model.img:test_image})
    out = output[0,3]
    seg = np.argmax(out,axis=2)
    temp_acc = cal_acc(seg,my_scale(cut3[:,:,3]))
    if temp_acc != 0:
        acc_list.append(temp_acc)
    temp_iou = cal_iou(seg,my_scale(cut3[:,:,3]))
    if temp_iou != 0:
        iou_list.append(temp_iou)

In [28]:
np.mean(iou_list)

0.22543014601631825

In [11]:
surreal_data = pd.read_csv('surreal_data_table.csv')
surreal_data.head()

,Unnamed: 0,jpg,mat
0,0,surreal_data/images\0.jpg,surreal_data/masks\0.mat
1,1,surreal_data/images\1.jpg,surreal_data/masks\1.mat
2,2,surreal_data/images\10.jpg,surreal_data/masks\10.mat
3,3,surreal_data/images\100.jpg,surreal_data/masks\100.mat
4,4,surreal_data/images\1000.jpg,surreal_data/masks\1000.mat


In [14]:
i = 18
test_img = image.load_img(surreal_data['jpg'][i])
test_img = image.img_to_array(test_img)
test_mask = scipy.io.loadmat(surreal_data['mat'][i])['mask']
cut1,cut2,cut3 = random_put_into_256map(test_img,test_mask)
plt.figure()
plt.imshow(cut3[:,:,0:3]/256)
plt.show()
test_image = np.expand_dims(cut3[:,:,0:3],0)
output = sess.run(model.output,feed_dict={model.img:test_image})
out = output[0,3]
seg = np.argmax(out,axis=2)
plt.figure()
plt.imshow(seg)
plt.show()

In [35]:
acc_list=[]
iou_list=[]
for i in range(3450,3500):
    test_img = image.load_img(surreal_data['jpg'][i])
    test_img = image.img_to_array(test_img)
    test_mask = scipy.io.loadmat(surreal_data['mat'][i])['mask']
    cut1,cut2,cut3 = random_put_into_256map(test_img,test_mask)
    test_image = np.expand_dims(cut3[:,:,0:3],0)
    output = sess.run(model.output,feed_dict={model.img:test_image})
    out = output[0,3]
    seg = np.argmax(out,axis=2)
    temp_acc = cal_acc(seg,my_scale(cut3[:,:,3]))
    if temp_acc != 0:
        acc_list.append(temp_acc)
    temp_iou = cal_iou(seg,my_scale(cut3[:,:,3]))
    if temp_iou != 0:
        iou_list.append(temp_iou)

In [37]:
np.mean(acc_list)

0.7487492679283918

In [56]:
img = image.img_to_array(image.load_img('me.JPG',target_size=[256,256]))
plt.figure()
plt.imshow(img/255)
plt.show()
test_image = np.expand_dims(img,0)
output = sess.run(model.output,feed_dict={model.img:test_image})
out = output[0,3]
seg = np.argmax(out,axis=2)
plt.figure()
plt.imshow(seg)
plt.show()